In [21]:
from urllib2 import Request, urlopen, HTTPError
from urlparse import urlunparse, urlparse
import json 
import pandas as pd
from matplotlib import pyplot as plt
import requests

In [22]:
from bs4 import BeautifulSoup
import urllib

<b>step 1</b>

In [44]:
r = urllib.urlopen('https://itunes.apple.com/us/genre/ios-books/id6018?mt=8').read()
soup = BeautifulSoup(r)
print type(soup)

<class 'bs4.BeautifulSoup'>


In [45]:
all_categories = soup.find_all("div", class_="nav")
category_url = all_categories[0].find_all(class_ = "top-level-genre")
categories_url = pd.DataFrame()
for itm in category_url:
    category = itm.get_text()
    url = itm.attrs['href']
    d = {'category':[category], 'url':[url]}
    df = pd.DataFrame(d)
    categories_url = categories_url.append(df, ignore_index = True)
print categories_url

                  category                                                url
0                    Books  https://itunes.apple.com/us/genre/ios-books/id...
1                 Business  https://itunes.apple.com/us/genre/ios-business...
2                 Catalogs  https://itunes.apple.com/us/genre/ios-catalogs...
3                Education  https://itunes.apple.com/us/genre/ios-educatio...
4            Entertainment  https://itunes.apple.com/us/genre/ios-entertai...
5                  Finance  https://itunes.apple.com/us/genre/ios-finance/...
6             Food & Drink  https://itunes.apple.com/us/genre/ios-food-dri...
7                    Games  https://itunes.apple.com/us/genre/ios-games/id...
8         Health & Fitness  https://itunes.apple.com/us/genre/ios-health-f...
9                Lifestyle  https://itunes.apple.com/us/genre/ios-lifestyl...
10  Magazines & Newspapers  https://itunes.apple.com/us/genre/ios-magazine...
11                 Medical  https://itunes.apple.com/us/genre/io

In [25]:
categories_url['url'][0]

'https://itunes.apple.com/us/genre/ios-books/id6018?mt=8'

<b>step2</b>

In [46]:
def extract_apps(url):
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r)
    apps = soup.find_all("div", class_="column")
    apps_link = apps[0].find_all('a')
    column_first = pd.DataFrame()
    for itm in apps_link:
        app_name = itm.get_text()
        url = itm.attrs['href']
        d = {'category':[app_name], 'url':[url]}
        df = pd.DataFrame(d)
        column_first = column_first.append(df, ignore_index = True)
    apps_link2 = apps[1].find_all('a')
    column_second = pd.DataFrame()
    for itm in apps_link2:
        app_name = itm.get_text()
        url = itm.attrs['href']
        d = {'category':[app_name], 'url':[url]}
        df = pd.DataFrame(d)
        column_second = column_second.append(df, ignore_index = True)
    apps_link3 = apps[2].find_all('a')
    column_last = pd.DataFrame()
    for itm in apps_link3:
        app_name = itm.get_text()
        url = itm.attrs['href']
        d = {'category':[app_name], 'url':[url]}
        df = pd.DataFrame(d)
        column_last = column_last.append(df, ignore_index = True)
    Final_app_link = pd.DataFrame()
    Final_app_link = Final_app_link.append(column_first, ignore_index = True)
    Final_app_link = Final_app_link.append(column_second, ignore_index = True)
    Final_app_link = Final_app_link.append(column_last, ignore_index = True)
    return Final_app_link

In [47]:
app_url = pd.DataFrame()
for itm in categories_url['url']:
    apps = extract_apps(itm)
    app_url = app_url.append(apps, ignore_index = True)

In [48]:
app_url['url'][0]

'https://itunes.apple.com/us/app/kindle-read-ebooks-magazines-textbooks/id302584613?mt=8'

<b>step3</b>

In [303]:
def get_content(url):
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r)
    des = soup.find_all('div', id = "content")
    apps = soup.find_all("div", class_="lockup product application")
    rate = soup.find_all("div", class_="extra-list customer-ratings")
    dic = []
    global app_name, descript, link, price, category, current_rate, current_count, total_count, total_rate
    for itm in des:
        try:
            descript = itm.find_all('div',{'class':"product-review"})[0].get_text().strip().split('\n')[2].encode('utf-8')
        except:
            descript = ''
        try:
            app_name = itm.find_all('div',{'class':"left" })[0].get_text().split('\n')[1]
        except:
            app_name = ''
    for itm in apps:
        category = itm.find_all('span',{'itemprop':"applicationCategory" })[0].get_text()
        price = itm.find_all('div',{'class':"price" })[0].get_text() 
        link = itm.a["href"]
    for itm in rate:
        try:
            current_rate = itm.find_all('span',{'itemprop':"ratingValue"})[0].get_text()
        except:
            current_rate = ''
        try:
            current_count = itm.find_all('span',{'itemprop':"reviewCount"})[0].get_text()
        except:
            current_count = ''
        try:
            total_count = itm.find_all('span',{'class':"rating-count"})[1].get_text()
        except:
            try:
                total_count = itm.find_all('span',{'class':"rating-count"})[0].get_text()
            except:
                total_count = ''
        try:
            total_rate = itm.find_all('div', class_="rating",itemprop = False)[0]['aria-label'].split(',')[0]
        except:
            total_rate = ''
    try:
        com = soup.find_all("div", class_="customer-reviews")
        dic_com = {}
        num = []
        rat = com[0].find_all("div",class_ = "rating")
        for itm in rat:
            a = itm['aria-label']
            num.append(a)
        for i in range(len(num)):
            star = com[0].find_all("div", class_="rating")[i]['aria-label']
            user = com[0].find_all("span", class_='user-info')[i].get_text()
            comment = com[0].find_all("p", class_="content")[i].get_text()
            diction = {user:(star,comment)}
            dic_com.update(diction)
    except:
        dic_com = {}        
    dic.append({'app':app_name,'link':link, 'price':price,'category':category,'current rating':current_rate, 'current reviews':current_count,'total rating':total_rate,'total reviews':total_count,'description':descript,'comment':dic_com})
    dic = pd.DataFrame(dic)
    return dic

In [ ]:
full_content_new = pd.DataFrame()
for itm in app_url['url']:
    content = get_content(itm)
    full_content_new = full_content_new.append(content, ignore_index = True)

In [347]:
full_content_new

,app,category,comment,current rating,current reviews,description,link,price,total rating,total reviews
0,"Kindle – Read eBooks, Magazines & Textbooks",Books,{u'by Lorring ...,2.59184,49 Ratings,Turn your iPhone or iPad into a Kindle with th...,https://itunes.apple.com/us/app/kindle-read-eb...,Free,3 and a half stars,245660 Ratings
1,"Audible – audio books, original series & podcasts",Books,{u'by Smuckitelly ...,4.64774,1902 Ratings,"Welcome to Audible. We’re an Amazon company, a...",https://itunes.apple.com/us/app/audible-audio-...,Free,4 stars,100843 Ratings
2,Wattpad - Free Books and eBook Reader,Books,{u'by RedPandaWorld ...,4.50867,173 Ratings,"Discover Wattpad:At Wattpad, we’re connecting ...",https://itunes.apple.com/us/app/wattpad-free-b...,Free,4 and a half stars,236679 Ratings
3,"NOOK - Read Books, Magazines, Newspapers & Comics",Books,{u'by Morgan737364737 ...,4.3384,1052 Ratings,"Get the FREE NOOK Reading App for your iPad, i...",https://itunes.apple.com/us/app/nook-read-book...,Free,4 stars,55576 Ratings
4,OverDrive – Library eBooks and Audiobooks,Books,{u'by Miss_Pink_7878 ...,3.73077,338 Ratings,"Borrow eBooks, audiobooks, and streaming video...",https://itunes.apple.com/us/app/overdrive-libr...,Free,4 stars,62987 Ratings
5,HOOKED - Chat Stories,Books,{u'by Meggo B ...,4.40088,454 Ratings,HOOKED lets you read amazing chat stories FREE...,https://itunes.apple.com/us/app/hooked-chat-st...,Free,4 and a half stars,27827 Ratings
6,Goodreads – Book Recommendations and Reviews,Books,{u'by Zanykfc ...,4.46969,132 Ratings,Discover and share books you love on Goodreads...,https://itunes.apple.com/us/app/goodreads-book...,Free,4 and a half stars,52946 Ratings
7,Google Play Books - Books & Comics,Books,{u'by attyjames ...,4.53365,1040 Ratings,With Google Play Books for iOS start reading t...,https://itunes.apple.com/us/app/google-play-bo...,Free,3 stars,11419 Ratings
8,"Scribd - Books, audiobooks, magazines, documents",Books,{},,,"Are you ready for access to the best books, au...",https://itunes.apple.com/us/app/scribd-books-a...,Free,4 and a half stars,11987 Ratings
9,Audiobooks,Books,{u'by 123Meier ...,3.5303,66 Ratings,Thousands of FREE audiobooks await. Whether yo...,https://itunes.apple.com/us/app/audiobooks/id3...,Free,3 stars,49153 Ratings


In [ ]:
full_content_new.to_csv('app.csv',encoding='utf-8')